In [ ]:
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from libsync.spotify.spotify_api_utils import (
    get_spotify_song_details,
    get_user_playlists_details,
)
import nest_asyncio
from libsync.utils.string_utils import (
    get_spotify_uri_from_id,
    pretty_print_spotify_track,
)

# hardcode playlist ID
playlist_id = "0w888XrmxysfqkgJrs5X0P"

# get secrets
load_dotenv()

# spotify auth
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=auth_manager)

# allow nested async (asyncio inside jupyter)
nest_asyncio.apply()

# get playlist details
playlist_details = get_user_playlists_details([playlist_id])

# get song details
uris = [get_spotify_uri_from_id(id) for id in playlist_details[playlist_id]]
songs = get_spotify_song_details(uris)

In [ ]:
# DO STUFF WITH THE DATA (no network calls here)

# pretty print
for song, details in songs.items():
    print(pretty_print_spotify_track(details))